In [16]:
%load_ext autoreload
%autoreload 2

import os
from time import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.utils.cit import CIT

import vis
import softclustering as sc
import socceraction.spadl as spadl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
datafolder = "data"
fifa2018h5 = os.path.join(datafolder, "spadl-fifa2018.h5")
games = pd.read_hdf(fifa2018h5, key="games")
with pd.HDFStore(fifa2018h5) as store:
    actions = []  #list of DataFrames
    for game in tqdm(games.itertuples()):
        game_action = store[f"actions/game_{game.game_id}"]
        game_action = spadl.play_left_to_right(game_action, game.home_team_id)
        game_action["is_home"] = game_action["team_id"] == game.home_team_id
        actions.append(game_action)
    actions = pd.concat(actions)
    actions.drop("original_event_id", axis=1, inplace=True)
    actions = pd.merge(actions, spadl.config.actiontypes_df(), how="left")
    actions = sc.consolidate(actions)
    actions = sc.add_noise(actions)
    actions = sc.remove_outliers(actions, True)
    actions["angle"] = np.arctan2(actions.end_y - actions.start_y, actions.end_x - actions.start_x)
    actions["cos_angle"] = np.cos(actions["angle"])
    actions["sin_angle"] = np.sin(actions["angle"])


0it [00:00, ?it/s]
21it [00:00, 202.16it/s]
64it [00:00, 209.98it/s]


Remove 536 out of 128484 datapoints.


In [20]:
no_dir_action_types = set(actions[actions["angle"] == 0.0]["type_name"])
action_types = set(actions["type_name"]) - no_dir_action_types - {"shot_penalty"}

for action in ["pass"]:
    print(f"Processing {action}...\n")
    mask = (actions["type_name"]==action) & (actions["team_id"]==771)
    columns = ["start_x", "start_y","cos_angle", "sin_angle"]
    #columns = ["start_x", "start_y","angle"]
    data = actions[mask][columns]
    cg = pc(data.to_numpy(),
                alpha=0.05,
                indep_test="fastkci",
                kernelX='Gaussian',
                kernelY='Gaussian',
                est_width='empirical',
            )
    pyd = GraphUtils.to_pydot(cg.G, labels=columns)
    pyd.write_pdf(f'plots/DAG_aatest_{action}.pdf')

Processing pass...




Depth=2, working on node 3: 100%|██████████| 4/4 [00:06<00:00,  1.55s/it]


In [21]:
no_dir_action_types = set(actions[actions["angle"] == 0.0]["type_name"])
action_types = set(actions["type_name"]) - no_dir_action_types - {"shot_penalty"}

for action in action_types:
    print(f"Processing {action}...\n")
    mask = (actions["type_name"]==action) & (actions["team_id"]==771)
    columns = ["start_x", "start_y","cos_angle", "sin_angle"]
    #columns = ["start_x", "start_y","angle"]
    data = actions[mask][columns]
    cg = pc(data.to_numpy(),
                alpha=0.05,
                indep_test="fastkci",
                kernelX='Gaussian',
                kernelY='Gaussian',
                est_width='empirical',
            )
    pyd = GraphUtils.to_pydot(cg.G, labels=columns)
    pyd.write_pdf(f'plots/DAG2_{action}.pdf')

Processing corner...




Depth=0, working on node 0:  25%|██▌       | 1/4 [00:00<00:00, 1406.54it/s]/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  data = stats.zscore(data, ddof=1, axis=0)
/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/causallearn/utils/FastKCI/FastKCI.py:470: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly iden

Processing pass...




Depth=2, working on node 3: 100%|██████████| 4/4 [00:08<00:00,  2.13s/it]


Processing freekick...



...


Depth=2, working on node 3: 100%|██████████| 4/4 [00:00<00:00, 21.18it/s]  


Processing dribble...




Depth=2, working on node 2:  75%|███████▌  | 3/4 [00:09<00:03,  3.15s/it]/Users/jgv/PycharmProjects/soccer-infogeo/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(

Depth=2, working on node 3: 100%|██████████| 4/4 [00:15<00:00,  3.91s/it]


Processing clearance...




Depth=1, working on node 3: 100%|██████████| 4/4 [00:00<00:00,  7.75it/s]
